Detección de caras con webcam

In [10]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import numpy as np
import math
import dlib


En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [15]:
# Inicializar el detector de caras y puntos faciales
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Tamaño del píxel para pixelar la cara
pixel_size = 10

# Variable para realizar o desactivar la censura
censor_mode = 0  # 0: Sin censura, 1: Censura de ojos, 2: Pixelación de cara, 3: Censura gaussiana


payaso_img = cv2.imread("Jose_el_payaso.png", -1)

def calculate_angle(A, B, C):
    """Calcula el ángulo entre tres puntos A, B y C. B es el vértice del ángulo."""
    BA = (A[0] - B[0], A[1] - B[1])
    BC = (C[0] - B[0], C[1] - B[1])
    cosine_angle = (BA[0] * BC[0] + BA[1] * BC[1]) / (math.sqrt(BA[0]**2 + BA[1]**2) * math.sqrt(BC[0]**2 + BC[1]**2))
    angle = math.acos(cosine_angle)
    return math.degrees(angle)

def mouth_aspect_ratio(mouth):
    angle = calculate_angle(mouth[0], mouth[6], mouth[9])  # Puntos 48, 54 y 57
    ANGLE_THRESHOLD = 32  # Ajusta este valor según sea necesario
    print("Angulo= " + str(angle))
    return angle > ANGLE_THRESHOLD

def detect_smile(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    smile_detected = False

    for face in faces:
        landmarks = predictor(gray, face)
        
        # Extracción de puntos de la boca
        mouth_points = []
        for n in range(48, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            mouth_points.append((x, y))
            #cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        # Detección de la sonrisa
        if mouth_points:
            smile_detected = mouth_aspect_ratio(mouth_points)
            print("Sonriendo = " + str(smile_detected))
        

    return smile_detected, image

def place_image(large_image, small_image, x_offset, y_offset):
    for c in range(0, 3):
        large_image[y_offset:y_offset + small_image.shape[0], x_offset:x_offset + small_image.shape[1], c] = \
            small_image[:, :, c] * (small_image[:, :, 3] / 255.0) + \
            large_image[y_offset:y_offset + small_image.shape[0], x_offset:x_offset + small_image.shape[1], c] * \
            (1.0 - small_image[:, :, 3] / 255.0)

font = cv2.FONT_HERSHEY_SIMPLEX        
modos = ["Nada", "Censura de ojos", "Pixelacion de cara", "Censura de cara", "Payaso cuando sonries :D"]
# Aquí se puede ajustar la posición y tamaño del payaso según sea necesario
resized_payaso = cv2.resize(payaso_img, (100, 100))  # Redimensiona según sea necesario

while True:
    ret, frame = cap.read()

    # Detectar caras en el marco
    faces = detector(frame)

    for face in faces:
        # Obtener landmarks faciales
        shape = predictor(frame, face)

        if censor_mode == 1:
            margen_horizontal = 25  # Margen adicional en cada lado horizontalmente
            margen_vertical = 10     # Margen adicional en cada lado verticalmente

            left_eye_x = min(shape.part(36).x, shape.part(39).x) - margen_horizontal
            right_eye_x = max(shape.part(42).x, shape.part(45).x) + margen_horizontal
            top_eye_y = min(shape.part(37).y, shape.part(38).y, shape.part(43).y, shape.part(44).y) - margen_vertical
            bottom_eye_y = max(shape.part(41).y, shape.part(40).y, shape.part(47).y, shape.part(46).y) + margen_vertical

            # Asegurarse de que las coordenadas no sean menores que 0 o mayores que las dimensiones de la imagen
            left_eye_x = max(left_eye_x, 0)
            right_eye_x = min(right_eye_x, frame.shape[1])
            top_eye_y = max(top_eye_y, 0)
            bottom_eye_y = min(bottom_eye_y, frame.shape[0])

            # Dibujar el rectángulo relleno
            frame[top_eye_y:bottom_eye_y, left_eye_x:right_eye_x] = (0, 0, 0)
            cv2.putText(frame, modos[censor_mode], (50, 20), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        elif censor_mode == 2:
            # Pixelar la cara
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_roi = frame[y:y + h, x:x + w]
            face_roi = cv2.resize(face_roi, (w // pixel_size, h // pixel_size))
            face_roi = cv2.resize(face_roi, (w, h), interpolation=cv2.INTER_LINEAR)
            frame[y:y + h, x:x + w] = face_roi
            cv2.putText(frame, modos[censor_mode], (50, 20), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        elif censor_mode == 3:
            # Aplicar censura gaussiana
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_roi = frame[y:y + h, x:x + w]
            face_roi = cv2.GaussianBlur(face_roi, (0, 0), sigmaX=30)
            frame[y:y + h, x:x + w] = face_roi
            cv2.putText(frame, modos[censor_mode], (50, 20), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        elif censor_mode == 4:
            is_smiling, processed_frame = detect_smile(frame)
            if is_smiling:
                faces = detector(frame)
                for face in faces:
                    landmarks = predictor(frame, face)
                    
                    # Asumiendo que el landmark 33 es la punta de la nariz
                    nose_tip = landmarks.part(30)
                    x, y = nose_tip.x, nose_tip.y

                    
                    place_image(frame, resized_payaso, x - 50, y - 50)
            cv2.putText(frame, modos[censor_mode], (50, 20), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            #print("Smiling:", is_smiling)  # Imprimir el estado de la sonrisa

    # Mostrar el marco con la censura correspondiente
    cv2.imshow('Censor Mode', frame)

    key = cv2.waitKey(1)

    if key == 27:  # Presiona Esc para salir
        break
    elif key == ord('d'):  # Presiona "d" para alternar entre los modos de censura
        censor_mode = (censor_mode + 1) % 5
    elif key == ord('e'):  # Presiona "d" para alternar entre los modos de censura
        censor_mode = (censor_mode - 1) % 5

cap.release()
cv2.destroyAllWindows()

Angulo= 17.241459398939973
Sonriendo = False
Angulo= 20.830387555272605
Sonriendo = False
Angulo= 24.145541960421657
Sonriendo = False
Angulo= 19.181462174375948
Sonriendo = False
Angulo= 20.77225468204584
Sonriendo = False
Angulo= 24.798305693259525
Sonriendo = False
Angulo= 24.062052224227827
Sonriendo = False
Angulo= 20.13630342824813
Sonriendo = False
Angulo= 18.434948822922017
Sonriendo = False
Angulo= 20.460700290901066
Sonriendo = False
Angulo= 24.724421391009034
Sonriendo = False
Angulo= 29.839428702069043
Sonriendo = True
Angulo= 29.700646000181667
Sonriendo = True
Angulo= 29.054604099077153
Sonriendo = True
Angulo= 29.86153355417953
Sonriendo = True
Angulo= 30.256437163529263
Sonriendo = True
Angulo= 29.57783868126134
Sonriendo = True
Angulo= 30.256437163529263
Sonriendo = True
Angulo= 30.256437163529263
Sonriendo = True
Angulo= 31.42956561483853
Sonriendo = True
Angulo= 30.5518107520446
Sonriendo = True
Angulo= 30.362663284253223
Sonriendo = True
Angulo= 32.15229514128234
So